<a href="https://colab.research.google.com/github/trongtn2110/CS114.L21/blob/master/Colab/data_discovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATA DISCOVERY**

## 1. Import các thư viện cần thiết, đọc dữ liệu

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
url1="https://raw.githubusercontent.com/Uranium1901/CS114.L21/main/Data_Discovery/wecode.csv"
url2="https://raw.githubusercontent.com/Uranium1901/CS114.L21/main/Data_Discovery/score_training.csv"
wecode_data=pd.read_csv(url1)
score_data=pd.read_csv(url2)

## 2. Xử lí trên file wecode

In [ ]:
wecode_data

,id,assignment_id,problem_id,user_id,is_final,status,pre_score,coefficient,file_name,language_id,created_at,updated_at
0,4,0,3,4,1,SCORE,10000,100,solution-0,2,2020-10-30 10:58:11,2020-10-30 10:58:19
1,5,0,4,4,1,SCORE,10000,100,solution-1,2,2020-10-30 11:07:09,2020-10-30 11:07:17
2,6,0,5,4,0,Compilation Error,0,100,solution-2,2,2020-10-30 11:08:01,2020-10-30 11:23:36
3,7,0,5,4,0,Compilation Error,0,100,solution-3,2,2020-10-30 11:11:41,2020-10-30 11:11:42
4,8,0,5,4,0,Compilation Error,0,100,solution-4,2,2020-10-30 11:15:06,2020-10-30 11:15:07
...,...,...,...,...,...,...,...,...,...,...,...,...
264601,266667,0,25,1796,0,SCORE,0,100,solution-3977,4,2021-04-28 09:34:20,2021-04-28 09:34:27
264602,266668,0,25,1796,0,Compilation Error,0,100,solution-3978,2,2021-04-28 09:35:07,2021-04-28 09:35:08
264603,266669,0,25,1796,1,SCORE,10000,100,solution-3979,4,2021-04-28 09:35:15,2021-04-28 09:35:21
264604,266670,0,150,255,1,SCORE,6250,100,solution-3980,2,2021-05-11 01:34:57,2021-05-11 01:35:28


In [ ]:
# Loại bỏ các cột dữ liệu không cần thiết
wecode_data.drop(['language_id','updated_at','file_name','status'],axis=1)
wecode_data.sort_values('user_id')

,id,assignment_id,problem_id,user_id,is_final,status,pre_score,coefficient,file_name,language_id,created_at,updated_at
13940,14008,0,81,1,1,Compilation Error,0,100,solution-1791,2,2020-11-08 16:50:50,2020-11-08 16:50:50
84860,84971,0,370,1,1,SCORE,10000,100,solution-2367,2,2020-12-01 01:59:52,2020-12-01 02:00:05
18060,18146,0,139,4,1,SCORE,10000,100,solution-1862,2,2020-11-10 14:45:31,2020-11-10 14:45:40
18218,18304,0,140,4,0,SCORE,10000,100,solution-1865,2,2020-11-10 15:21:27,2020-11-10 15:23:54
18361,18447,0,141,4,0,SCORE,9000,100,solution-1869,2,2020-11-10 15:56:39,2020-11-10 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
236800,236913,0,638,1868,0,SCORE,10000,100,solution-3318,2,2021-01-22 15:38:10,2021-01-22 15:38:20
236788,236901,0,638,1868,0,SCORE,8000,100,solution-3317,2,2021-01-22 15:33:55,2021-01-22 15:34:04
234349,234462,0,639,1868,1,SCORE,10000,100,solution-3308,2,2021-01-21 14:21:23,2021-01-21 14:21:33
211358,211471,0,585,1868,1,SCORE,10000,100,solution-3089,2,2021-01-14 17:44:24,2021-01-14 17:44:34


In [ ]:
#Số problem trong mỗi assignment
sub_wecode1= pd.DataFrame(wecode_data[['user_id','assignment_id','problem_id','is_final','pre_score']])
PperA=pd.DataFrame(sub_wecode1.groupby('assignment_id').problem_id.nunique())
PperA.columns=['total_problem']
PperA

,total_problem
assignment_id,
0,352
5,4
7,10
8,8
10,7
...,...
376,10
377,7
381,4


In [ ]:
# Sinh viên hoàn thành bao nhiêu problem trong mỗi asignment
solved_temp=sub_wecode1[(sub_wecode1['is_final']==1) & (sub_wecode1['pre_score']==10000)]
temp=pd.DataFrame(solved_temp.groupby(['user_id','assignment_id'])['problem_id'].count())
temp.columns=['solved']
solved=pd.merge(temp,PperA,on=('assignment_id'),right_index=True)
solved['Percent']=(solved['solved']*100)/solved['total_problem']
solved

,,solved,total_problem,Percent
user_id,assignment_id,,,
1,0,1,352,0.284091
4,0,37,352,10.511364
6,0,83,352,23.579545
11,0,8,352,2.272727
15,0,28,352,7.954545
...,...,...,...,...
1847,173,15,15,100.000000
1849,173,15,15,100.000000
1850,173,13,15,86.666667


In [ ]:
#Phần trăm trung bình sinh viên đó giải được các problem
result1= solved.groupby("user_id").agg({"Percent": np.mean})
result1

,Percent
user_id,
1,0.284091
4,10.511364
5,12.777778
6,25.637626
11,13.636364
...,...
1860,40.625000
1862,35.000000
1864,46.746032


In [ ]:
#tổng hợp các dữ liệu khai thác được
temp=wecode_data.groupby(['user_id']).count()
wecode=pd.DataFrame()
wecode['x']=wecode_data['user_id'].nunique()
del wecode['x']
#số asignment sinh viên tham giả giải
wecode['assignment']=wecode_data.groupby(['user_id']).assignment_id.nunique()
#số problem sinh viên tham gia giải
wecode['problem']=wecode_data.groupby(['user_id']).problem_id.nunique()
#số problem sinh viene giải được
wecode['is_solved']=solved_temp.groupby('user_id')['is_final'].count()
#trung bình số lần submit đến khi problem được giải quyết
wecode['mean_tries_to_solve']=temp['id']/wecode['is_solved']
#Phần trăm trung bình sinh viên đó giải được các problem
wecode['Percent_solved']=result1['Percent']
wecode

,assignment,problem,is_solved,mean_tries_to_solve,Percent_solved
user_id,,,,,
1,1,2,1.0,2.000000,0.284091
4,1,41,37.0,1.459459,10.511364
5,4,5,3.0,2.000000,12.777778
6,4,88,85.0,1.529412,25.637626
7,1,2,NaN,NaN,NaN
...,...,...,...,...,...
1860,10,43,28.0,2.714286,40.625000
1862,2,6,4.0,3.000000,35.000000
1864,5,63,28.0,6.678571,46.746032


## 3.Xử lí trên file core_training 

In [ ]:
score_data

,id,QT,GK,TH,CK,TBM,TBTL
0,22,10,9,8.5,8.5,8.8,8.52
1,24,10,8,10,9.5,9.4,8.41
2,27,10,10,10,10,10,8.67
3,28,10,9.5,7.5,9.5,9.2,7.39
4,29,10,9.5,5,8.5,8.2,8.05
...,...,...,...,...,...,...,...
970,1857,5,4.5,10,4,6,6.03
971,1859,8,5,10,1.5,5.4,5.06
972,1862,5,6.5,1,7.5,5.1,4.71
973,1865,6.5,,3,7.5,,6.89


In [ ]:
# Lấy nhưng cột dữ liệu cần thiết
new_score_data=pd.DataFrame(score_data[['id','TH','TBM']])
new_score_data.columns=['user_id','TH','TBM']
new_score_data

,user_id,TH,TBM
0,22,8.5,8.8
1,24,10,9.4
2,27,10,10
3,28,7.5,9.2
4,29,5,8.2
...,...,...,...
970,1857,10,6
971,1859,10,5.4
972,1862,1,5.1
973,1865,3,


In [ ]:
# Kết 2 bảng wecode và score lọc ra các user_id không có dữ liệu điểm
result=pd.merge(wecode,new_score_data,on='user_id')
result

,user_id,assignment,problem,is_solved,mean_tries_to_solve,Percent_solved,TH,TBM
0,24,15,78,72.0,2.277778,64.812925,10,9.4
1,27,2,5,3.0,4.333333,11.253157,10,10
2,29,13,80,72.0,2.763889,72.906469,5,8.2
3,34,11,84,81.0,2.691358,82.500000,9,
4,35,25,147,145.0,2.048276,88.657538,6,6.2
...,...,...,...,...,...,...,...,...
848,1856,5,31,23.0,2.782609,55.000000,7,6.7
849,1857,6,34,37.0,1.486486,79.166667,10,6
850,1859,7,27,3.0,10.333333,75.000000,10,5.4
851,1862,2,6,4.0,3.000000,35.000000,1,5.1
